In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sbn
import tensorflow as tf
import shap
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import make_classification
import csv


In [2]:
df = pd.read_csv("BERPublicsearch.csv",sep='\t', error_bad_lines=False, warn_bad_lines=True, quoting=csv.QUOTE_NONE)

Columns (61,62,63,64,65,66,71,73,97,98,139,144,150,155,161,166) have mixed types.Specify dtype option on import or set low_memory=False.


In [3]:
df = df.fillna(0)
pd.set_option('display.max_columns', None) 

In [4]:
#String değer içeren sütunlar siliniyor.
df = df.select_dtypes(exclude=["object"])

In [5]:
x = df

In [6]:
x

,Year_of_Construction,BerRating,GroundFloorArea(sq m),UValueWall,UValueRoof,UValueFloor,UValueWindow,UvalueDoor,WallArea,RoofArea,FloorArea,WindowArea,DoorArea,NoStoreys,CO2Rating,HSMainSystemEfficiency,TGDLEdition,MPCDERValue,HSEffAdjFactor,HSSupplHeatFraction,HSSupplSystemEff,WHMainSystemEff,WHEffAdjFactor,SupplSHFuel,SupplWHFuel,SHRenewableResources,WHRenewableResources,NoOfChimneys,NoOfOpenFlues,NoOfFansAndVents,NoOfFluelessGasFires,FanPowerManuDeclaredValue,HeatExchangerEff,PercentageDraughtStripped,NoOfSidesSheltered,PermeabilityTestResult,TempAdjustment,HeatSystemControlCat,HeatSystemResponseCat,NoCentralHeatingPumps,NoOilBoilerHeatingPumps,NoGasBoilerHeatingPumps,GroundFloorUValue,DistributionLosses,WaterStorageVolume,DeclaredLossFactor,TempFactorUnadj,TempFactorMultiplier,InsulationThickness,CombiBoilerAddLoss,ElecConsumpKeepHot,ApertureArea,ZeroLossCollectorEff,CollectorHeatLossCoEff,AnnualSolarRadiation,OvershadingFactor,SolarStorageVolume,VolumeOfPreHeatStore,ElectricityConsumption,gsdHSSupplHeatFraction,gsdHSSupplSystemEff,DistLossFactor,CHPUnitHeatFraction,CHPSystemType,CHPElecEff,CHPHeatEff,CHPFuelType,SupplHSFuelTypeID,SolarHeatFraction,DeliveredLightingEnergy,DeliveredEnergyPumpsFans,DeliveredEnergyMainWater,DeliveredEnergyMainSpace,PrimaryEnergyLighting,PrimaryEnergyPumpsFans,PrimaryEnergyMainWater,PrimaryEnergyMainSpace,CO2Lighting,CO2PumpsFans,CO2MainWater,CO2MainSpace,GroundFloorArea,GroundFloorHeight,FirstFloorArea,FirstFloorHeight,SecondFloorArea,SecondFloorHeight,ThirdFloorArea,ThirdFloorHeight,ThermalBridgingFactor,PredominantRoofTypeArea,LowEnergyLightingPercent,TotalDeliveredEnergy,DeliveredEnergySecondarySpace,DeliveredEnergySupplementaryWater,LivingAreaPercent,CO2SecondarySpace,CO2SupplementaryWater,PrimaryEnergySecondarySpace,PrimaryEnergySupplementaryWater,HESSchemeUpgrade,RoomInRoofArea,FirstEnergyTypeId,FirstEnerProdDelivered,FirstPartLTotalContribution,FirstEnerProdConvFactor,FirstEnerProdCO2EmissionFactor,FirstEnerConsumedDelivered,FirstEnerConsumedConvFactor,FirstEnerConsumedCO2EmissionFactor,SecondEnergyTypeId,SecondEnerProdDelivered,SecondPartLTotalContribution,SecondEnerProdConvFactor,SecondEnerProdCO2EmissionFactor,SecondEnerConsumedDelivered,SecondEnerConsumedConvFactor,SecondEnerConsumedCO2EmissionFactor,ThirdEnergyTypeId,ThirdEnerProdDelivered,ThirdPartLTotalContribution,ThirdEnerProdConvFactor,ThirdEnerProdCO2EmissionFactor,ThirdEnerConsumedDelivered,ThirdEnerConsumedConvFactor,ThirdEnerConsumedCO2EmissionFactor,FirstBoilerFuelType,FirstHeatGenPlantEff,FirstPercentageHeat,SecondBoilerFuelType,SecondHeatGenPlantEff,SecondPercentageHeat,ThirdBoilerFuelType,ThirdHeatGenPlantEff,ThirdPercentageHeat,SolarSpaceHeatingSystem,TotalPrimaryEnergyFact,TotalCO2Emissions,FirstWallArea,FirstWallUValue,FirstWallAgeBandId,FirstWallTypeId,SecondWallArea,SecondWallUValue,SecondWallAgeBandId,SecondWallTypeId,ThirdWallArea,ThirdWallUValue,ThirdWallAgeBandId,ThirdWallTypeId,prob_smarea_error_0corr,prob_smarea_error_100corr,RER,RenewEPnren,RenewEPren,CPC,EPC
0,1992,192.95,169.700,0.27,0.27,0.56,2.80,0.00,121.73,106.22,109.36,35.41,0.00,2,48.73,70.00,0,44.87,0.95,0.1,65.0,70.00,0.95,15.0,1.0,2.0,2.0,0.0,2.0,1.0,0.0,0.00,0.0,100.0,2.0,0.000,0.6,1,2,1,1,0,0.52,468.508,300.0,0.0,0.6,1.30,30.0,0.0,0.0,3.6,0.82,3.29,1005.0,1.0,200.0,0.0,75.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1194.603,230.000,5223.141,18898.173,3225.428,621.00,5745.455,20787.990,768.130,147.890,1420.694,5140.303,109.360,2.60,60.34,2.34,0.0,0.0,0.0,0.0,0.15,48.03,44.0,27694.171,2148.254,0.000,10.8427,792.706,0.000,2363.079,0.000,1,0.0,1.0,0.00,0.0,0.00,0.000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,105.120,0.27,7.0,5.0,16.61,0.270,9.0,5.0,0.00,0.00,0.0,0.0,0.000000e+00,0.0000,0.000,0.00,0.00,0.000,0.000
1,1980,296.97,132.100,0.26,0.13,0.48,2.85,2.36,101.75,132.10,132.10,37.86,3.70,1,76.36,65.00,0,63.37,0.95,0.1,30.0,65.00,0.95,15.0,28.0,2.0,2.0,2.0,0.0,2.0,0.

In [7]:
#SHAP uygulaması için feature değerleri alınıyor.
features = x.columns

In [8]:
features

Index(['Year_of_Construction', 'BerRating', 'GroundFloorArea(sq m)',
       'UValueWall', 'UValueRoof', 'UValueFloor', 'UValueWindow', 'UvalueDoor',
       'WallArea', 'RoofArea',
       ...
       'ThirdWallUValue', 'ThirdWallAgeBandId', 'ThirdWallTypeId',
       'prob_smarea_error_0corr', 'prob_smarea_error_100corr', 'RER',
       'RenewEPnren', 'RenewEPren', 'CPC', 'EPC'],
      dtype='object', length=157)

In [9]:
#Verisetinde hedef verisi olmadığı için KMeans algoritması ile hedef verileri oluşturuldu.
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2, random_state=0)
kmeans.fit(x)

labels = kmeans.predict(x)

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, labels, test_size = 0.30, random_state = 15)

In [11]:
x_train

,Year_of_Construction,BerRating,GroundFloorArea(sq m),UValueWall,UValueRoof,UValueFloor,UValueWindow,UvalueDoor,WallArea,RoofArea,FloorArea,WindowArea,DoorArea,NoStoreys,CO2Rating,HSMainSystemEfficiency,TGDLEdition,MPCDERValue,HSEffAdjFactor,HSSupplHeatFraction,HSSupplSystemEff,WHMainSystemEff,WHEffAdjFactor,SupplSHFuel,SupplWHFuel,SHRenewableResources,WHRenewableResources,NoOfChimneys,NoOfOpenFlues,NoOfFansAndVents,NoOfFluelessGasFires,FanPowerManuDeclaredValue,HeatExchangerEff,PercentageDraughtStripped,NoOfSidesSheltered,PermeabilityTestResult,TempAdjustment,HeatSystemControlCat,HeatSystemResponseCat,NoCentralHeatingPumps,NoOilBoilerHeatingPumps,NoGasBoilerHeatingPumps,GroundFloorUValue,DistributionLosses,WaterStorageVolume,DeclaredLossFactor,TempFactorUnadj,TempFactorMultiplier,InsulationThickness,CombiBoilerAddLoss,ElecConsumpKeepHot,ApertureArea,ZeroLossCollectorEff,CollectorHeatLossCoEff,AnnualSolarRadiation,OvershadingFactor,SolarStorageVolume,VolumeOfPreHeatStore,ElectricityConsumption,gsdHSSupplHeatFraction,gsdHSSupplSystemEff,DistLossFactor,CHPUnitHeatFraction,CHPSystemType,CHPElecEff,CHPHeatEff,CHPFuelType,SupplHSFuelTypeID,SolarHeatFraction,DeliveredLightingEnergy,DeliveredEnergyPumpsFans,DeliveredEnergyMainWater,DeliveredEnergyMainSpace,PrimaryEnergyLighting,PrimaryEnergyPumpsFans,PrimaryEnergyMainWater,PrimaryEnergyMainSpace,CO2Lighting,CO2PumpsFans,CO2MainWater,CO2MainSpace,GroundFloorArea,GroundFloorHeight,FirstFloorArea,FirstFloorHeight,SecondFloorArea,SecondFloorHeight,ThirdFloorArea,ThirdFloorHeight,ThermalBridgingFactor,PredominantRoofTypeArea,LowEnergyLightingPercent,TotalDeliveredEnergy,DeliveredEnergySecondarySpace,DeliveredEnergySupplementaryWater,LivingAreaPercent,CO2SecondarySpace,CO2SupplementaryWater,PrimaryEnergySecondarySpace,PrimaryEnergySupplementaryWater,HESSchemeUpgrade,RoomInRoofArea,FirstEnergyTypeId,FirstEnerProdDelivered,FirstPartLTotalContribution,FirstEnerProdConvFactor,FirstEnerProdCO2EmissionFactor,FirstEnerConsumedDelivered,FirstEnerConsumedConvFactor,FirstEnerConsumedCO2EmissionFactor,SecondEnergyTypeId,SecondEnerProdDelivered,SecondPartLTotalContribution,SecondEnerProdConvFactor,SecondEnerProdCO2EmissionFactor,SecondEnerConsumedDelivered,SecondEnerConsumedConvFactor,SecondEnerConsumedCO2EmissionFactor,ThirdEnergyTypeId,ThirdEnerProdDelivered,ThirdPartLTotalContribution,ThirdEnerProdConvFactor,ThirdEnerProdCO2EmissionFactor,ThirdEnerConsumedDelivered,ThirdEnerConsumedConvFactor,ThirdEnerConsumedCO2EmissionFactor,FirstBoilerFuelType,FirstHeatGenPlantEff,FirstPercentageHeat,SecondBoilerFuelType,SecondHeatGenPlantEff,SecondPercentageHeat,ThirdBoilerFuelType,ThirdHeatGenPlantEff,ThirdPercentageHeat,SolarSpaceHeatingSystem,TotalPrimaryEnergyFact,TotalCO2Emissions,FirstWallArea,FirstWallUValue,FirstWallAgeBandId,FirstWallTypeId,SecondWallArea,SecondWallUValue,SecondWallAgeBandId,SecondWallTypeId,ThirdWallArea,ThirdWallUValue,ThirdWallAgeBandId,ThirdWallTypeId,prob_smarea_error_0corr,prob_smarea_error_100corr,RER,RenewEPnren,RenewEPren,CPC,EPC
1030966,2002,208.65,123.91,0.55,0.36,0.44,2.80,3.00,93.45,109.81,76.16,20.72,1.85,2,52.90,79.00,0,42.29,0.95,0.1,30.0,79.00,0.95,8.0,28.0,1.0,1.0,1.0,0.0,3.0,0.0,0.00,0.0,100.0,3.0,0.000,0.0,1,1,1,1,0,0.00,292.023,0.0,0.000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,272.481,230.000,3681.509,12730.573,566.760,478.40,4049.660,14003.630,111.445,94.070,1001.370,3462.716,76.16,2.50,0.00,0.00,0.0,0.0,0.0,0.0,0.15,109.813,0.0,21672.672,3570.292,1187.817,14.4100,1399.554,485.817,4284.350,2470.660,0,47.75,1.0,0.0,0.00,0.00,0.000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,93.450,0.55,9.0,10.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,2.500000e-03,0.002500,0.000,24332.40,0.00,1.663,1.355
631143,1998,242.34,99.78,0.55,0.40,0.41,3.10,3.06,86.90,53.49,53.36,18.47,2.50,2,49.87,73.00,0,38.59,0.95,0.1,30.0,73.00,0.95,8.0,28.0,2.0,2.0,1.0,0.0,5.0

In [12]:

scaler = MinMaxScaler()
scaler.fit(x_train)
scaler.fit(x_test)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [13]:
#Conv2D sınıfı 4 boyutlu diziler istediği için boyutlar değiştiriliyor.
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

x_train = x_train.reshape((771946, 157, 1, 1))
y_train = y_train.reshape((771946, 1))
x_test = x_test.reshape((330834, 157,1, 1))
y_test = y_test.reshape((330834, 1))

(771946, 157)
(771946,)
(330834, 157)
(330834,)


In [14]:
model = Sequential()

model.add(Conv2D(filters = 12, kernel_size = (3,3), input_shape = (157, 1, 1), activation='relu', padding="same"))
model.add(Conv2D(filters = 18, kernel_size = (3,3), activation='relu', padding="same"))
model.add(MaxPooling2D(pool_size=(2,2), padding="same"))

model.add(Conv2D(filters = 32, kernel_size = (3,3), activation='relu', padding="same"))
model.add(MaxPooling2D(pool_size=(2,2), padding="same"))


model.add(Flatten())
model.add(Dense(64))
model.add(Dense(32))
model.add(Dense(1))

In [15]:
model.compile(optimizer = "adam", loss='binary_crossentropy', 
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, validation_data=(x_test, y_test),epochs = 2,batch_size=200)

Epoch 1/2
3860/3860 [==============================] - 290s 75ms/step - loss: 0.2938 - accuracy: 0.8807 - val_loss: 0.2352 - val_accuracy: 0.9128
Epoch 2/2
 717/3860 [====>.........................] - ETA: 4:36 - loss: 0.2724 - accuracy: 0.9074

In [ ]:
loss = model.history.history["loss"]
val_loss = model.history.history["val_loss"]

In [ ]:
epochs = np.arange(1, len(loss) + 1)
plt.figure(figsize=(12,6))
plt.plot(epochs, loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.xlabel('Epochs', fontsize=14)
plt.ylabel('Loss', fontsize=14)
plt.title('Training vs Validation Loss')

plt.legend()

In [ ]:
#SHAP metodunda kullanılabilmesi için veri boyutu küçültülüyor.
ref_trainX, ref_testX, ref_trainY, ref_testY = train_test_split(x_train, y_train, test_size=0.6)    

In [ ]:
ref_trainX, ref_testX, ref_trainY, ref_testY = train_test_split(ref_trainX, ref_trainY, test_size=0.6)  
ref_trainX, ref_testX, ref_trainY, ref_testY = train_test_split(ref_trainX, ref_trainY, test_size=0.9)  

In [ ]:
ref_trainX, ref_testX, ref_trainY, ref_testY = train_test_split(ref_trainX, ref_trainY, test_size=0.6)  
ref_trainX, ref_testX, ref_trainY, ref_testY = train_test_split(ref_trainX, ref_trainY, test_size=0.9)

In [ ]:
ref_trainX, ref_testX, ref_trainY, ref_testY = train_test_split(ref_trainX, ref_trainY, test_size=0.6)
ref_trainX, ref_testX, ref_trainY, ref_testY = train_test_split(ref_trainX, ref_trainY, test_size=0.9)

In [ ]:
ref_trainX.shape

In [ ]:
import warnings

explainer = shap.DeepExplainer(model, ref_trainX)
shap_values = explainer.shap_values(ref_testX)

In [ ]:
#shap.summary_plot'ta kullanılacak olan shap_values numpy dizisi olmak zorunda.
shap_values = np.array(shap_values)

In [ ]:
shap_values.shape

In [ ]:
#Kullanılacak olan iki dizide aynı boyutta olmalı, (numberof_samples, numberof_features) şeklinde
shap_values = shap_values.reshape((178,157))

In [ ]:
ref_testX = ref_testX.reshape((178,157))

In [ ]:
shap.summary_plot(
   shap_values, features=ref_testX, feature_names=features, plot_type="bar")

In [ ]:
shap.summary_plot(shap_values, ref_testX.astype("float"), feature_names=features)

In [ ]:
trainLoss, trainAcc = model.evaluate(x_train, y_train, verbose = 0)

In [ ]:
testLoss, testAcc = model.evaluate(x_test, y_test, verbose = 0)

In [ ]:
predict = model.predict(x_test)

In [ ]:
predict

In [ ]:
#0 ve 1 arasında olması beklenen veriler daha büyük çıktığı için veriler normalize ediliyor.
scaler.fit(predict)
predict = scaler.transform(predict)

In [ ]:
predict = pd.Series(predict.reshape(330834,))

In [ ]:
#Verilerin 0 ya da 1 olması istendiği için veriler yuvarlanıyor.
y_pred = predict.round()

In [ ]:
y_pred

In [ ]:
y_true = pd.Series(y_test.reshape(330834,))

In [ ]:
y_true

In [ ]:
#y_pred ve y_true aynı çeşitte değerler içermeli.
y_pred = y_pred.astype('int32')

In [ ]:
print(classification_report(y_true, y_pred, labels=[0, 1], 
                              target_names=['Regular', 'Thief']))

In [ ]:

plt.figure(figsize=(8, 8))
false_positive_rate, true_positive_rate, _ = roc_curve(y_true, y_pred, pos_label=1)
roc_auc = roc_auc_score(y_true, y_pred)
plt.plot(false_positive_rate, true_positive_rate, lw=2, label='ROC Curve (area = %0.2f)' % roc_auc)
plt.xlabel('False Positive Rate', fontsize=14)
plt.ylabel('True Positive Rate', fontsize=14)
plt.title('ROC Curve')
plt.legend(loc='lower right')


In [ ]:
matrix = confusion_matrix(y_true, y_pred)


plt.figure(figsize=(8, 8))
sbn.heatmap(matrix, annot=True, fmt='.5g', cmap='Blues', 
        xticklabels=['Predicted Regular', 'Predicted Thief'], 
        yticklabels=['True Regular', 'True Thief'])
plt.title('Confusion Matrix ')